<a href="https://colab.research.google.com/github/Ru1chi/First-Project/blob/main/recipe_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
# Load recipe data
recipe_data = pd.read_excel('/content/recipes.csv.xlsx')



In [ ]:
recipe_data

,RecipeID,RecipeName,Ingredients,Instructions,Cuisine,DietaryPreferences
0,1,Spaghetti Bolognese,"Spaghetti, Ground Beef, Tomato Sauce, Garlic, ...",Cook spaghetti. Cook beef with garlic and onio...,Italian,"High Protein, Low Carb"
1,2,Vegan Tacos,"Tortillas, Black Beans, Corn, Avocado, Tomato,...","Warm tortillas. Fill with beans, corn, avocado...",Mexican,"Vegan, Gluten Free"
2,3,Chicken Salad,"Chicken Breast, Lettuce, Tomato, Cucumber, Oli...","Grill chicken. Mix with chopped lettuce, tomat...",American,"High Protein, Low Fat"


In [ ]:
# Describe the data and print it
print(recipe_data.describe())
print(recipe_data)

# Print the shape of the data
print(recipe_data.shape)



       RecipeID
count       3.0
mean        2.0
std         1.0
min         1.0
25%         1.5
50%         2.0
75%         2.5
max         3.0
   RecipeID           RecipeName  \
0         1  Spaghetti Bolognese   
1         2          Vegan Tacos   
2         3        Chicken Salad   

                                         Ingredients  \
0  Spaghetti, Ground Beef, Tomato Sauce, Garlic, ...   
1  Tortillas, Black Beans, Corn, Avocado, Tomato,...   
2  Chicken Breast, Lettuce, Tomato, Cucumber, Oli...   

                                        Instructions   Cuisine  \
0  Cook spaghetti. Cook beef with garlic and onio...   Italian   
1  Warm tortillas. Fill with beans, corn, avocado...   Mexican   
2  Grill chicken. Mix with chopped lettuce, tomat...  American   

       DietaryPreferences  
0  High Protein, Low Carb  
1      Vegan, Gluten Free  
2   High Protein, Low Fat  
(3, 6)


In [ ]:
# Selecting features of our choice
features = ['Instructions', 'Cuisine', 'DietaryPreferences']

# Filling the null values with empty strings
for feature in features:
    recipe_data[feature] = recipe_data[feature].fillna(' ')

# Combine all the selected features into a single string
combined_features = recipe_data['Instructions'] + ' ' + recipe_data['Cuisine'] + ' ' + recipe_data['DietaryPreferences']



In [ ]:
# Converting text to feature vectors using TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features)

# Print the feature vectors
print(feature_vector)

# Print the shape of the feature vectors
print(feature_vector.shape)



  (0, 6)	0.23360373424918143
  (0, 24)	0.17766166113862597
  (0, 31)	0.17766166113862597
  (0, 19)	0.17766166113862597
  (0, 20)	0.23360373424918143
  (0, 33)	0.23360373424918143
  (0, 26)	0.17766166113862597
  (0, 32)	0.23360373424918143
  (0, 36)	0.13797012860507832
  (0, 0)	0.23360373424918143
  (0, 30)	0.23360373424918143
  (0, 2)	0.35532332227725194
  (0, 16)	0.23360373424918143
  (0, 41)	0.13797012860507832
  (0, 5)	0.23360373424918143
  (0, 34)	0.23360373424918143
  (0, 9)	0.46720746849836287
  (1, 15)	0.25585529205196594
  (1, 17)	0.25585529205196594
  (1, 39)	0.25585529205196594
  (1, 25)	0.25585529205196594
  (1, 37)	0.25585529205196594
  (1, 29)	0.25585529205196594
  (1, 23)	0.25585529205196594
  (1, 35)	0.25585529205196594
  :	:
  (1, 10)	0.25585529205196594
  (1, 4)	0.25585529205196594
  (1, 14)	0.25585529205196594
  (1, 38)	0.25585529205196594
  (1, 40)	0.25585529205196594
  (1, 36)	0.15111225709707723
  (1, 41)	0.15111225709707723
  (2, 13)	0.2563720506124583
  (2, 1)	0.

In [ ]:
# Calculate the cosine similarity
similarity = cosine_similarity(feature_vector)

# Print the similarity matrix
print(similarity)



[[1.         0.04169796 0.27051348]
 [0.04169796 1.         0.10658272]
 [0.27051348 0.10658272 1.        ]]


In [ ]:
# Creating a list of all ingredients in the dataset
list_of_all_ingredients = recipe_data['Ingredients'].tolist()



In [ ]:
# Preprocess ingredients in the dataset: tokenize and normalize
def preprocess_ingredients(ingredient_list):
    return [ingredient.strip().lower() for ingredient in ingredient_list.split(',')]

recipe_data['ProcessedIngredients'] = recipe_data['Ingredients'].apply(preprocess_ingredients)



In [ ]:
# Function to match user-given ingredients with the dataset
def match_ingredients(user_ingredients, recipe_data):
    user_ingredients = [ingredient.lower().strip() for ingredient in user_ingredients]
    matched_recipes = recipe_data[recipe_data['ProcessedIngredients'].apply(lambda x: any(ingredient in x for ingredient in user_ingredients))]
    return matched_recipes



In [ ]:
# Ask user for ingredients
user_input = input("Enter the ingredients you want in your recipe (separated by commas): ")
ingredient_names = [ingredient.strip() for ingredient in user_input.split(',')]



Enter the ingredients you want in your recipe (separated by commas): tomato,onion


In [ ]:
# Find matching recipes
matched_recipes = match_ingredients(ingredient_names, recipe_data)



In [ ]:
if not matched_recipes.empty:
    # Get indices of matched recipes
    indices = matched_recipes.index.tolist()

    # Print the matched recipes
    print(f"Recipes containing any of the ingredients '{', '.join(ingredient_names)}':")
    for idx in indices:
        print(recipe_data.loc[idx, 'RecipeName'])

    # Find the similarity scores for matched recipes and recommend similar recipes
    for idx in indices:
        similarity_scores = list(enumerate(similarity[idx]))
        sorted_similar_recipes = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

        print(f"\nRecipes similar to '{recipe_data.loc[idx, 'RecipeName']}':")
        i = 1
        for recipe in sorted_similar_recipes[:10]:  # top 10 similar recipes
            index = recipe[0]
            if index != idx:
                print(i, '.', recipe_data['RecipeName'].iloc[index])
                i += 1
else:
    print("No recipes found containing the given ingredients.")


Recipes containing any of the ingredients 'tomato, onion':
Spaghetti Bolognese
Vegan Tacos
Chicken Salad

Recipes similar to 'Spaghetti Bolognese':
1 . Chicken Salad
2 . Vegan Tacos

Recipes similar to 'Vegan Tacos':
1 . Chicken Salad
2 . Spaghetti Bolognese

Recipes similar to 'Chicken Salad':
1 . Spaghetti Bolognese
2 . Vegan Tacos


In [ ]:
import pickle

In [ ]:

filename='recipe_recommendation.sav'



In [ ]:


# Create a dictionary to store both objects
save_data = {'vectorizer': vectorizer, 'similarity': similarity}

# Save the dictionary containing both objects
with open('recipe_recommendation.sav', 'wb') as file:
    pickle.dump(save_data, file)


In [ ]:
loaded_model=pickle.load(open('recipe_recommendation.sav','rb'))